In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

print(train.shape, test.shape)

(13645, 22) (8745, 20)


# feature preprocessing

In [3]:
train.head()

,EmpID,EmpName,LanguageOfCommunication,Age,Gender,JobProfileIDApplyingFor,HighestDegree,DegreeBranch,GraduatingInstitute,LatestDegreeCGPA,...,CurrentCTC,ExpectedCTC,MartialStatus,EmpScore,CurrentDesignation,CurrentCompanyType,DepartmentInCompany,TotalLeavesTaken,BiasInfluentialFactor,FitmentPercent
0,11041,John,English,35,Male,JR85289,B.Tech,Electrical,Tier 1,7,...,21,26,Married,5,SSE,Enterprise,Design,20,YearsOfExperince,95.40
1,15079,William,English,26,Male,JR87525,B.Tech,Artificial Intelligence,Tier 3,7,...,15,19,Married,5,BA,MidSized,Engineering,6,NaN,67.09
2,18638,James,English,36,Female,JR87525,PhD,Computer Science,Tier 1,6,...,15,24,Single,5,SDE,MidSized,Engineering,19,Gender,91.26
3,3941,Charles,English,29,Female,JR87525,BCA,Information Technology,Tier 2,5,...,16,24,Married,5,SDE,Startup,Product,16,Gender,72.29
4,5936,George,English,25,Male,JR70175,Dual M.Tech,Computer Science,Tier 3,8,...,24,32,Married,5,SDE,Enterprise,Engineering,10,DegreeBranch,86.34


In [4]:
train.columns

Index(['EmpID', 'EmpName', 'LanguageOfCommunication', 'Age', 'Gender',
       'JobProfileIDApplyingFor', 'HighestDegree', 'DegreeBranch',
       'GraduatingInstitute', 'LatestDegreeCGPA', 'YearsOfExperince',
       'GraduationYear', 'CurrentCTC', 'ExpectedCTC', 'MartialStatus',
       'EmpScore', 'CurrentDesignation', 'CurrentCompanyType',
       'DepartmentInCompany', 'TotalLeavesTaken', 'BiasInfluentialFactor',
       'FitmentPercent'],
      dtype='object')

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13645 entries, 0 to 13644
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   EmpID                    13645 non-null  int64  
 1   EmpName                  13645 non-null  object 
 2   LanguageOfCommunication  13645 non-null  object 
 3   Age                      13645 non-null  int64  
 4   Gender                   13645 non-null  object 
 5   JobProfileIDApplyingFor  13645 non-null  object 
 6   HighestDegree            13645 non-null  object 
 7   DegreeBranch             13645 non-null  object 
 8   GraduatingInstitute      13645 non-null  object 
 9   LatestDegreeCGPA         13645 non-null  int64  
 10  YearsOfExperince         13645 non-null  int64  
 11  GraduationYear           13645 non-null  int64  
 12  CurrentCTC               13645 non-null  int64  
 13  ExpectedCTC              13645 non-null  int64  
 14  MartialStatus         

In [6]:
print(train.shape)

train1 = train[[ 'LanguageOfCommunication', 'Age', 'Gender',
       'JobProfileIDApplyingFor', 'HighestDegree', 'DegreeBranch',
       'GraduatingInstitute', 'LatestDegreeCGPA', 'YearsOfExperince',
       'CurrentCTC', 'MartialStatus',
       'EmpScore', 'CurrentDesignation', 'CurrentCompanyType',
       'DepartmentInCompany', 'TotalLeavesTaken']].copy()
print(train1.shape)

train1['GraduatingInstitute'] = train['GraduatingInstitute'].map({'Tier 1':3,'Tier 2':2, 'Tier 3':1})
train1['ExpectedHike'] = (train['ExpectedCTC']-train['CurrentCTC'])/train['CurrentCTC']*100
print(train1.shape)
train1.head()

(13645, 22)
(13645, 16)
(13645, 17)


,LanguageOfCommunication,Age,Gender,JobProfileIDApplyingFor,HighestDegree,DegreeBranch,GraduatingInstitute,LatestDegreeCGPA,YearsOfExperince,CurrentCTC,MartialStatus,EmpScore,CurrentDesignation,CurrentCompanyType,DepartmentInCompany,TotalLeavesTaken,ExpectedHike
0,English,35,Male,JR85289,B.Tech,Electrical,3,7,12,21,Married,5,SSE,Enterprise,Design,20,23.809524
1,English,26,Male,JR87525,B.Tech,Artificial Intelligence,1,7,3,15,Married,5,BA,MidSized,Engineering,6,26.666667
2,English,36,Female,JR87525,PhD,Computer Science,3,6,6,15,Single,5,SDE,MidSized,Engineering,19,60.000000
3,English,29,Female,JR87525,BCA,Information Technology,2,5,6,16,Married,5,SDE,Startup,Product,16,50.000000
4,English,25,Male,JR70175,Dual M.Tech,Computer Science,1,8,2,24,Married,5,SDE,Enterprise,Engineering,10,33.333333


In [7]:
target_col = train['BiasInfluentialFactor'].fillna('Unknown')
target_col.head()

0    YearsOfExperince
1             Unknown
2              Gender
3              Gender
4        DegreeBranch
Name: BiasInfluentialFactor, dtype: object

# Creating X and Y

In [8]:
X = pd.get_dummies(train1, drop_first=True).values
y = target_col.copy().values
print(X.shape,y.shape)

(13645, 46) (13645,)


# encoding label

In [9]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_en = le.fit_transform(y)

In [10]:
le.classes_

array(['CurrentCompanyType', 'DegreeBranch', 'EmpScore', 'Ethinicity',
       'Gender', 'HighestDegree', 'LatestDegreeCGPA', 'MartialStatus',
       'Unknown', 'YearsOfExperince'], dtype=object)

# Data Split

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train,y_val = train_test_split(X,y_en, test_size=.2, stratify=y_en,random_state=123)
print(X_train.shape, X_val.shape, y_train.shape,y_val.shape)

(10916, 46) (2729, 46) (10916,) (2729,)


# Scaling

In [19]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_train_scaled[0]

array([ 0.59349849, -1.58196293,  0.66956113,  1.00470499, -0.29050469,
        0.58194186, -0.00508413, -0.34723673, -0.73016852, -0.33323154,
        0.79356173, -0.27239308, -0.22634634, -0.22945998, -0.72281026,
        1.72279181, -0.17738534, -0.29775933, -0.31686468, -0.30517271,
        2.71144814, -0.33051001, -0.22679325, -0.42153182, -0.09949451,
       -0.35863448, -0.21863641, -0.73046345, -0.39278143, -0.17294069,
        2.64409062, -0.52505307,  1.5921602 , -0.14865187, -0.10181616,
       -0.37978257, -0.33272227, -0.81130176, -0.14572846,  2.00298217,
        1.92330341, -0.93315582, -0.69422192,  1.18150809, -0.23297755,
       -0.31335146])

In [23]:
X_train_scaled[1]

array([ 0.07618069, -0.20892426,  0.66956113,  0.42019999,  0.40376565,
       -0.3720268 ,  0.19895389, -0.65548557, -0.73016852,  3.00091645,
       -1.26014141, -0.27239308, -0.22634634,  4.35805851, -0.72281026,
       -0.58045319, -0.17738534, -0.29775933, -0.31686468, -0.30517271,
       -0.36880661, -0.33051001, -0.22679325, -0.42153182, -0.09949451,
       -0.35863448, -0.21863641, -0.73046345, -0.39278143,  5.78232922,
       -0.37820186, -0.52505307,  1.5921602 , -0.14865187, -0.10181616,
        2.63308555, -0.33272227, -0.81130176, -0.14572846, -0.49925557,
       -0.51993877,  1.07163239, -0.69422192,  1.18150809, -0.23297755,
       -0.31335146])

# Model

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import time

# rf

In [38]:
clf_rf = RandomForestClassifier(n_estimators=250,n_jobs=-1)

In [39]:
clf_rf.fit(X_train,y_train)

RandomForestClassifier(n_estimators=250, n_jobs=-1)

In [40]:
y_pred = clf_rf.predict(X_val)

In [41]:
print(classification_report(y_val,y_pred))
print(confusion_matrix(y_val,y_pred))

              precision    recall  f1-score   support

           0       0.82      0.26      0.39       235
           1       0.86      0.92      0.89       572
           2       0.78      0.93      0.85       132
           3       0.00      0.00      0.00        66
           4       0.85      1.00      0.92       521
           5       0.81      0.97      0.89       116
           6       0.60      0.06      0.11        52
           7       0.59      0.36      0.44        56
           8       0.63      0.69      0.66       667
           9       0.85      0.97      0.91       312

    accuracy                           0.78      2729
   macro avg       0.68      0.62      0.61      2729
weighted avg       0.76      0.78      0.75      2729

[[ 60  15   0   0  21   7   0   0 125   7]
 [  1 527   0   0   0   0   0   0  41   3]
 [  0   0 123   0   0   0   0   0   8   1]
 [  0   0  27   0   0   0   0   6  33   0]
 [  0   0   0   0 521   0   0   0   0   0]
 [  0   0   0   0   0 113 

# With cross val

In [64]:
class_weight={6: 250,
 3: 500,
 0: 250,
 7:100,
 8:90
 }

In [65]:
clf_lgb = LGBMClassifier(verbose=2, n_estimators=3000, objective='multiclass',
                         n_jobs=-1)

In [66]:
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits)
ls_accuracy=[]
ls_scores=[]
ls_confusion_mat=[]

start=time.time()
for ind_train, ind_val in skf.split(X,y_en):
    X_train = X[ind_train,:]
    y_train = y_en[ind_train,]
    X_val = X[ind_val,:]
    y_val = y_en[ind_val,]
    
    print('---------------------------------------\nfitting...')
    clf_lgb.fit(X_train,y_train, early_stopping_rounds=300,
               eval_set=[(X_val, y_val)])
    
    print('predicting...')
    y_pred = clf_lgb.predict(X_val)
    
    ls_accuracy.append(accuracy_score(y_val,y_pred))
    ls_scores.append(classification_report(y_val,y_pred))
    ls_confusion_mat.append(classification_report(y_val,y_pred))
stop=time.time()
print(f'\n\ntime: {round((stop-start),2)} s')
print(f'acc: {ls_accuracy}')
print(f'Mean accuracy: {np.mean(ls_accuracy)}')
print(f'Std accuracy: {np.std(ls_accuracy)}')

---------------------------------------
fitting...
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.160682
[LightGBM] [Debug] init for col-wise cost 0.000026 seconds, init for row-wise cost 0.002849 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007330 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 298
[LightGBM] [Info] Number of data points in the train set: 10916, number of used features: 46
[LightGBM] [Info] Start training from score -2.453169
[LightGBM] [Info] Start training from score -1.562989
[LightGBM] [Info] Start training from score -3.028889
[LightGBM] [Info] Start training from score -3.729640
[LightGBM] [Info] Start training from score -1.655940
[LightGBM] [Info] Start training from score -3.155947
[LightGBM] [Info] Start training from score -3.9

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[25]	valid_0's multi_logloss: 0.560825
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[26]	valid_0's multi_logloss: 0.557736
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 1

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[39]	valid_0's multi_logloss: 0.54342
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with 

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[54]	valid_0's multi_logloss: 0.543432
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree wit

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[68]	valid_0's multi_logloss: 0.549184
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with

...

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[295]	valid_0's multi_logloss: 0.82163
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 18 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tre

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[308]	valid_0's multi_logloss: 0.837031
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 15 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[309]	valid_0's multi_logloss: 0.838187
[LightGBM] [Debug] Trained a tree with leaves = 31 and de

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 14 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[322]	valid_0's multi_logloss: 0.85566
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Warning] No further 

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 8
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 13 and depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[335]	valid_0's multi_logloss: 0.873491
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tr

predicting...


time: 61.88 s
acc: [0.7984609747160132, 0.7929644558446317, 0.7907658482960791, 0.7936973250274826, 0.7944301942103335]
Mean accuracy: 0.794063759618908
Std accuracy: 0.0025174883162063398


In [67]:
for i in ls_scores:
    print(i)

              precision    recall  f1-score   support

           0       0.69      0.27      0.39       235
           1       0.91      0.92      0.92       571
           2       0.78      0.95      0.86       131
           3       0.07      0.02      0.02        66
           4       0.87      0.99      0.92       521
           5       0.93      0.91      0.92       117
           6       0.34      0.25      0.29        52
           7       0.56      0.64      0.60        56
           8       0.68      0.73      0.70       667
           9       0.87      0.96      0.91       313

    accuracy                           0.80      2729
   macro avg       0.67      0.67      0.65      2729
weighted avg       0.78      0.80      0.78      2729

              precision    recall  f1-score   support

           0       0.65      0.32      0.43       235
           1       0.89      0.93      0.91       571
           2       0.79      0.95      0.86       132
           3       0.15 

# With smote

# Data Split

In [68]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train,y_val = train_test_split(X,y_en, test_size=.2, stratify=y_en,random_state=123)
print(X_train.shape, X_val.shape, y_train.shape,y_val.shape)

(10916, 46) (2729, 46) (10916,) (2729,)


# smote

In [69]:
[(c,i) for i,c in enumerate(list(train1.columns))]

[('LanguageOfCommunication', 0),
 ('Age', 1),
 ('Gender', 2),
 ('JobProfileIDApplyingFor', 3),
 ('HighestDegree', 4),
 ('DegreeBranch', 5),
 ('GraduatingInstitute', 6),
 ('LatestDegreeCGPA', 7),
 ('YearsOfExperince', 8),
 ('CurrentCTC', 9),
 ('MartialStatus', 10),
 ('EmpScore', 11),
 ('CurrentDesignation', 12),
 ('CurrentCompanyType', 13),
 ('DepartmentInCompany', 14),
 ('TotalLeavesTaken', 15),
 ('ExpectedHike', 16)]

In [93]:
dict(pd.Series(y_train).value_counts())

{8: 2669,
 1: 2286,
 4: 2084,
 9: 1249,
 0: 939,
 2: 527,
 5: 466,
 3: 262,
 7: 224,
 6: 210}

In [82]:
sampling_strategy_dict={8: 2669,
 1: 2600,
 4: 2400,
 9: 1900,
 0: 1900,
 2: 1600,
 5: 1600,
 3: 2500,
 7: 1600,
 6: 1500}

In [94]:
sampling_strategy_dict={
 6: 2100}

In [163]:
# categorical_features=np.array([0,2,3,4,5,6,10,12,13,14]),

In [95]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(sampling_strategy=sampling_strategy_dict,
              random_state=123, n_jobs=-1)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

print(X_train.shape,y_train.shape)
print(X_train_res.shape, y_train_res.shape)

(10916, 46) (10916,)
(12806, 46) (12806,)


In [96]:
pd.Series(y_train_res).value_counts()/len(X_train_res)*100

8    20.841793
1    17.851007
6    16.398563
4    16.273622
9     9.753241
0     7.332500
2     4.115258
5     3.638919
3     2.045916
7     1.749180
dtype: float64

In [88]:
clf_lgb = LGBMClassifier(verbose=2, n_estimators=3000, objective='multiclass',
                         n_jobs=-1)

In [97]:
clf_lgb.fit(X_train_res,y_train_res, early_stopping_rounds=200,
               eval_set=[(X_val, y_val)])

[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.856421
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.675699
[LightGBM] [Debug] init for col-wise cost 0.003885 seconds, init for row-wise cost 0.003783 seconds
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004632 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9343
[LightGBM] [Info] Number of data points in the train set: 12806, number of used features: 46
[LightGBM] [Info] Start training from score -2.612854
[LightGBM] [Info] Start training from score -1.723110
[LightGBM] [Info] Start training from score -3.190469
[LightGBM] [Info] Start training from score -3.889325
[LightGBM] [Info] Start training from score -1.815625
[LightGBM] [Info] Start training from score -3.313483
[LightGBM] [Info] Start training from score -1.807976
[LightGBM] [Info] Start training from score -4.046023
[LightGBM] [Info] S

[13]	valid_0's multi_logloss: 0.667538
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 19
[14]	valid_0's multi_logloss: 0.651162
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12


[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[26]	valid_0's multi_logloss: 0.569196
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree wit

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[44]	valid_0's multi_logloss: 0.558343
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree wit

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[56]	valid_0's multi_logloss: 0.563502
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[57]	valid_0's multi_logloss: 0.563553

...

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[194]	valid_0's multi_logloss: 0.700683
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree wi

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[209]	valid_0's multi_logloss: 0.718209
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[210]	valid_0's multi_logloss: 0.719527
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth =

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[223]	valid_0's multi_logloss: 0.735274
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[224]	valid_0's multi_logloss: 0.736459
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[240]	valid_0's multi_logloss: 0.757091
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 8
[LightGBM] [Debug] Trained a tree with

LGBMClassifier(n_estimators=3000, objective='multiclass', verbose=2)

In [98]:
y_pred = clf_lgb.predict(X_val)

print(classification_report(y_val,y_pred))
print(confusion_matrix(y_val,y_pred))

              precision    recall  f1-score   support

           0       0.68      0.32      0.43       235
           1       0.90      0.92      0.91       572
           2       0.77      0.94      0.85       132
           3       0.00      0.00      0.00        66
           4       0.85      0.99      0.92       521
           5       0.90      0.97      0.93       116
           6       0.39      0.25      0.31        52
           7       0.65      0.61      0.63        56
           8       0.66      0.70      0.68       667
           9       0.89      0.96      0.92       312

    accuracy                           0.79      2729
   macro avg       0.67      0.67      0.66      2729
weighted avg       0.77      0.79      0.77      2729

[[ 75   5   0   0  21   1   0   0 129   4]
 [  5 524   0   0   0   0   3   0  40   0]
 [  0   0 124   6   0   0   0   0   2   0]
 [  0   0  28   0   0   0   2   7  29   0]
 [  0   0   0   0 518   0   3   0   0   0]
 [  0   0   0   0   0 113 